In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
import gc
from csv import writer

In [9]:
!git clone https://github.com/matamorosjose95/GCIT-1.git

Cloning into 'GCIT-1'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 245 (delta 65), reused 41 (delta 18), pack-reused 136
Receiving objects: 100% (245/245), 129.79 KiB | 11.80 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [10]:
import sys
sys.path.append('/content/GCIT-1')
from GCIT import *
from utils import *
from modelos_gancito import *
from entrenamiento_gan_cit_0 import *

In [11]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [ ]:
Zdim = 25 # 25, 50, 100, 250, 500
dim_x = 1
dim_y = 1 
# fijar número de experimentos a realizar
n_experiments = 100 #faltan 900
# variable para guardar los pvalores de la función GCIT-1
p_vals= []
p_values = []
# determinar de manera aleatoria cual es el tipo de conjunto a evaluar
true = np.random.randint(0,2,n_experiments)
print('Probando test para '+str(Zdim)+' dimensiones de Z')
pbar = tqdm(total=n_experiments)
for n in range(n_experiments):
    ti = time.time()
    # CREAR DATASET
    if true[n]==0:
        data_type = 'CI' # conjunto condicionalmente independiente
    if true[n]==1:
        data_type = 'NI' # conjunto dependiente 
    x, y, z = generate_samples_random(size=500, sType=data_type,
                                      dx=dim_x, dy=dim_y, dz=Zdim,
                                      fixed_function='linear', dist_z='gaussian')
    # entrenar gancito y obtener modelo GAN entrenado
    zMI_loss, xMI_loss, G_loss, D_loss, G_net = training(x,y,z,'GCIT_InfoGAN', 2e-4) 
    # REALIZAR TEST
    n_samples = 1000
    p_vals = [] # para guardar los valores de los 5 estadísticos
    statistics = [correlation, mmd_squared, kolmogorov, wilcox, rdc]
    for stat in statistics:
        rho = []
        for sample in range(n_samples):
            v = torch.FloatTensor(1000, dim_v).cuda()
            x_hat = G_net.forward(torch.tensor(z).float().cuda(), v.uniform_(0,1).cuda())
            x_hat = x_hat.cpu().detach().numpy()
            rho.append(stat(x_hat, y))
        #print(rho)
        rho = np.array(rho)
        p_value = min(sum(rho < stat(x.reshape(len(x)),y)) / n_samples, sum(rho > stat(x.reshape(len(x)), y)) / n_samples)
        p_vals.append(p_value)
    p_vals = GCIT_1(x, y, z, statistic='all')
    del x, y, z
    pbar.update(1)
    # GUARDAR RESULTADOS
    append_list_as_row('dim%s_linear_gaussian_GANCIT0.csv'%(Zdim,), [true[n].tolist()] + p_vals)
    gc.collect()
p_values = np.array(p_values)
pbar.close()

Probando test para 25 dimensiones de Z
